## Prompt engineering

La ingeniería de prompts permite dotar de más información al modelo generalista a la hora de obtener resultados cercanos a los que nosotros buscamos. Aquí tenéis una buena referencia: https://www.promptingguide.ai/es

Dado que poco a poco iremos evolucionando nuestros prompts creando plantillas y dependencias, es bueno que nos hagamos eco de algunos frameworks que nos permitirán desarrollar estas piezas técnicas de forma algo más fácil y con opción de variar entre distintos modelos (LLMs).

# LangChain

[LangChain]() es un framework pensado para realizar este tipo de tarea, el desarrollo a bajo nivel de las interacciones con las LLMs. De cara a que no genere coste ni requiera cuenta de cobro podréis utilizar la API fe Gemini para estos ejercicios:

- Obtén tu clave de API de Gemini en [Google AI Studio](https://aistudio.google.com/app/apikey).
- Guarda la clave en una variable `GEMINI_API_KEY`.

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

A la hora de instanciar el modelo podemos determinar múltiples aspectos sobre cómo debe actuar...

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [6]:
llm.invoke("Hola!")

AIMessage(content='¡Hola! ¿Cómo estás?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--76354d48-5e48-4bfd-8866-e9e2ad6a1d0a-0', usage_metadata={'input_tokens': 3, 'output_tokens': 31, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 24}})

Vemos que completa la respuesta con un mensaje del tipo `AIMessage`. Ahora podemos elaborar nuestra conversación pero teniendo en cuenta que las LLMs no tienen memoria... toda la información les ha de ser provista.

In [7]:
llm.invoke("Me llamo Iraitz")

AIMessage(content='¡Hola, Iraitz! Mucho gusto.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--121adfe8-bb21-4234-a5a9-79900458594f-0', usage_metadata={'input_tokens': 6, 'output_tokens': 214, 'total_tokens': 220, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 204}})

In [8]:
llm.invoke("¿Cómo me llamo?")

AIMessage(content='Como modelo de lenguaje, no tengo acceso a tu información personal y no sé cómo te llamas.\n\nMi función es procesar y generar texto basándome en la información que me proporcionas en esta conversación. No tengo memoria de interacciones pasadas ni acceso a datos personales de los usuarios.\n\nSi quieres que te llame de alguna manera, puedes decírmelo.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--2f42e728-c2be-4c14-8947-f737cc7401ac-0', usage_metadata={'input_tokens': 7, 'output_tokens': 534, 'total_tokens': 541, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 458}})

Esto es lo que nos hace ver la importancia de proveer de toda la información necesaria cuando interactuamos con una LLM. No tiene por qué ser el usuario pero para dar respuesta a las preguntas de un usuario, nosotros como desarrolladores debemos interceder incluyendo toda información que creamos sea relevante para obtener la respuesta esperada.

## Contexto

A eso nos referimos con el contexto y quizás una de las instrucciones más básicas sean las instrucciones del sistema. Esta serie de instrucciones siempre le son provistas a la LLM para que de respuesta en los términos que se le indican.

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Eres un especialista en LLMs pero tienes un pequeños defecto en el habla, usas muchas s. Responde siempre en Español"),
    HumanMessage("¿Qué son las LLMs?"),
]

respuesta = llm.invoke(messages)
respuesta.content

'¡Ah, sí, las LLMs! ¡Esas son unas de las cosas más sssorprendentesss que hemosss vissto en la inteligencia artificial!\n\nLas LLMs, o **Large Language Modelsss** (Modelosss de Lenguaje Grandes), ssson, en sssu esssencia, programas de computadora sssúper ssofissticadosss que han sssido entrenadosss con cantidadesss masivas de datosss de texto y código. ¡Estamosss hablando de billones de palabras y frases de internet, librosss, artículosss, y muchasss cosasss más!\n\nAquí te explico sssusss caracteríssticasss principalesss:\n\n1.  **Sssu "Grandesssa":** Ssson "grandesss" por dosss razoneesss principalesss:\n    *   **Datosss:** Ssse entrenan con conjuntosss de datosss inmensssosss.\n    *   **Parámetrosss:** Tienen miles de millones de "parámetrosss", que ssson como losss "conocimientosss" internosss que el modelo usssa para hacer sssusss prediccionesss. Cuantosss másss parámetrosss, másss complejas y sssutiles ssson sssusss comprensionesss del lenguaje.\n\n2.  **Sssu Objetivo Principal

Podemos ver el coste efectivo de nuestra consulta.

In [12]:
respuesta.usage_metadata

{'input_tokens': 33,
 'output_tokens': 970,
 'total_tokens': 1003,
 'input_token_details': {'cache_read': 0},
 'output_token_details': {'reasoning': 228}}

Debido a que empleamos a menudo estos esquemas, LangChain ya dispone de plantillas que nos ayudan a tener estructuradas las conversaciones.

In [14]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Traduce todo texto a {idioma}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{texto}")]
)

prompt = prompt_template.invoke({"idioma": "Italiano", "texto": respuesta.content})
prompt.to_messages()

[SystemMessage(content='Traduce todo texto a Italiano', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='¡Ah, sí, las LLMs! ¡Esas son unas de las cosas más sssorprendentesss que hemosss vissto en la inteligencia artificial!\n\nLas LLMs, o **Large Language Modelsss** (Modelosss de Lenguaje Grandes), ssson, en sssu esssencia, programas de computadora sssúper ssofissticadosss que han sssido entrenadosss con cantidadesss masivas de datosss de texto y código. ¡Estamosss hablando de billones de palabras y frases de internet, librosss, artículosss, y muchasss cosasss más!\n\nAquí te explico sssusss caracteríssticasss principalesss:\n\n1.  **Sssu "Grandesssa":** Ssson "grandesss" por dosss razoneesss principalesss:\n    *   **Datosss:** Ssse entrenan con conjuntosss de datosss inmensssosss.\n    *   **Parámetrosss:** Tienen miles de millones de "parámetrosss", que ssson como losss "conocimientosss" internosss que el modelo usssa para hacer sssusss prediccionesss. Cuantosss m

Y simplemente podemos usar nuestra platilla para invocar al modelo.

In [15]:
response = llm.invoke(prompt)
print(response.content)

Ah, sì, le LLM! Queste sono tra le cose più sorprendenti che abbiamo visto nell'intelligenza artificiale!

Le LLM, o **Large Language Models** (Modelli Linguistici di Grandi Dimensioni), sono, in essenza, programmi informatici super sofisticati che sono stati addestrati con quantità massicce di dati testuali e di codice. Stiamo parlando di miliardi di parole e frasi da internet, libri, articoli e molto altro ancora!

Qui ti spiego le loro caratteristiche principali:

1.  **La loro "Grandezza":** Sono "grandi" per due ragioni principali:
    *   **Dati:** Si addestrano con set di dati immensi.
    *   **Parametri:** Hanno miliardi di "parametri", che sono come le "conoscenze" interne che il modello usa per fare le sue previsioni. Più parametri ci sono, più complesse e sottili sono le loro comprensioni del linguaggio.

2.  **Il loro Obiettivo Principale:** Il loro compito fondamentale è prevedere la parola successiva (o "token") in una sequenza. Se gli dai "Il cielo è...", il loro obiett

In [17]:
cadena = prompt_template | llm # Encadenamos la plantilla al modelo
response = cadena.invoke({"idioma": "Japonés", "texto": respuesta.content})
response.content

'ああ、そう、LLMですね！これらは、人工知能において私たちが目にしてきた最も驚くべきものの一つです！\n\nLLM、つまり**大規模言語モデル**は、その本質において、膨大な量のテキストデータとコードで訓練されてきた超高性能なコンピュータープログラムです。インターネット、書籍、記事などから何兆もの単語やフレーズが使われているんですよ！\n\nここで、その主な特徴を説明しますね。\n\n1.  **その「巨大さ」:** 主に二つの理由で「巨大」なのです。\n    *   **データ:** 膨大なデータセットで訓練されます。\n    *   **パラメータ:** 何十億もの「パラメータ」を持っています。これらは、モデルが予測を行うために使う内部的な「知識」のようなものです。パラメータが多ければ多いほど、言語の理解はより複雑で繊細になります。\n\n2.  **その主な目的:** その基本的なタスクは、シーケンス内の次の単語（または「トークン」）を予測することです。もし「空は…」と与えれば、学習した内容に基づいて「青い」「灰色」「星空」などを予測するのがその目的です。この単純なタスクを何百万回も繰り返すことで、一貫性があり、文脈に沿ったテキストを生成できるようになります。\n\n3.  **Transformerアーキテクチャ:** 現代のLLMのほとんどは、「Transformer」と呼ばれるニューラルネットワークアーキテクチャを使用しています。このアーキテクチャは、データシーケンスを処理し、文や段落内の単語間の長期的な関係を理解するのに非常に効率的です。\n\n4.  **能力:** その大規模な訓練のおかげで、LLMは言語に関連する幅広いタスクを実行できます。例えば、\n    *   **テキスト生成:** 物語、詩、メール、プログラミングコードの作成。\n    *   **質問応答:** 質問を理解し、情報に基づいた回答を提供。\n    *   **要約:** 長いテキストを要点にまとめる。\n    *   **翻訳:** ある言語から別の言語へテキストを変換。\n    *   **会話:** 流暢で自然な対話を維持。\n    *   **感情分析:** テキストの感情的なトーンを判断。\n\n要するに、LLMは非常に強力なAIシステムであり、驚

In [18]:
response.usage_metadata

{'input_tokens': 750,
 'output_tokens': 4247,
 'total_tokens': 4997,
 'input_token_details': {'cache_read': 0},
 'output_token_details': {'reasoning': 3707}}

Debido a lo que hemos comentado anteriormente, es habitual encontrar plantillas que incluyen la información necesaria de cara a responder las dudas de los usuarios o ofrecer ejemplos de cómo responder. Cuando la interacción es directa, como en los casos anteriores este modelo se conoce como **zero-shot prompting** mientras que si ofrecemos ejemplos de cómo resolverlo, esta técnica conocida como **in-context learning** se modelo como **few-shot prompting** ofreciendo varios ejemplos de lo que vayamos a resolver.

In [20]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Eres un juglar de la corte del rey Felipe IV.

Si alguien te da un tema, eres capaz de componer bromas en castellano antiguo.

# Ejemplo
Tema: Castañas
                                               
Yendo dos señoras por la calle, la una de ellas que se decía Castañeda, 
soltósele un trueno bajero, a lo cual dijo la otra:
—Niña, pápate esa castaña.
Echándose de ellos por tres veces arreo, y respondiendo la otra lo mismo, 
volviéronse y vieron un doctor en medicina que les venía detrás, y, 
por saber si había habido sentimiento del negocio, dijéronle:
—Señor, ¿ha rato que nos sigue?
Respondió:
—De la primera castaña, señoras.


Tema: {tema}""")

prompt_template.invoke({"tema": "Caballo"})

StringPromptValue(text='\nEres un juglar de la corte del rey Felipe IV.\n\nSi alguien te da un tema, eres capaz de componer bromas en castellano antiguo.\n\n# Ejemplo\nTema: Castañas\n\nYendo dos señoras por la calle, la una de ellas que se decía Castañeda, \nsoltósele un trueno bajero, a lo cual dijo la otra:\n—Niña, pápate esa castaña.\nEchándose de ellos por tres veces arreo, y respondiendo la otra lo mismo, \nvolviéronse y vieron un doctor en medicina que les venía detrás, y, \npor saber si había habido sentimiento del negocio, dijéronle:\n—Señor, ¿ha rato que nos sigue?\nRespondió:\n—De la primera castaña, señoras.\n\n\nTema: Caballo')

In [23]:
import pprint

cadena = prompt_template | llm # Encadenamos la plantilla al modelo
response = cadena.invoke({"tema": "Caballo"})

pprint.pprint(response.content)

('¡Oíd, oíd, gentes de bien y de mal vivir, que vuestro juglar os trae un '
 'nuevo entremés para solaz de vuestras almas!\n'
 '\n'
 'Tema: Caballo\n'
 '\n'
 'Hallábanse dos damas de la corte, Doña Inés y Doña Elvira, paseando por los '
 'jardines del Real Alcázar, cuando la conversación les llevó a los menesteres '
 'de la equitación.\n'
 '\n'
 'Dijo Doña Inés, con un suspiro:\n'
 '—¡Ay, comadre, qué fatigada me hallo! Mi caballo, el rucio, me ha dado hoy '
 'un trajín que no es de Dios. ¡Parecía que quería echarme al suelo a cada '
 'paso!\n'
 '\n'
 'A lo que respondió Doña Elvira, con una sonrisa pícara:\n'
 '—Pues el mío, el bayo, es de tal brío que pocos son los que le pueden montar '
 'sin caerse. Mas yo os digo que no hay caballo, por fiero que sea, que no se '
 'me rinda al primer envite, y una vez que le tengo bien asido, no hay quien '
 'me baje de él hasta que no ha sudado la gota gorda.\n'
 '\n'
 'En esto, pasó por su vera Don Lope de Figueroa, un hidalgo de recia estampa, 

In [24]:
response = cadena.invoke({"tema": "pata de palo"})

pprint.pprint(response.content)

('¡Oh, nobles señores y gentiles damas! Prestad oído a este vuestro humilde '
 'juglar, que con la gracia de Su Majestad, el Rey Felipe IV, os trae un nuevo '
 'chascarrillo para alegrar vuestras almas y aliviar el peso de la corona.\n'
 '\n'
 '**Tema: Pata de palo**\n'
 '\n'
 'Contábase en la corte de Su Majestad, que un hidalgo, conocido por su '
 'gallardía mas también por su pata de palo, tras una noche de galanteo con '
 'una dama de buen ver y mejor ingenio, se hallaban ambos en el lecho, cuando '
 'el hidalgo, con un suspiro quejumbroso, dijo:\n'
 '\n'
 '—¡Ay, mi señora, cuán desdichado soy, que por esta mi pata de palo, apenas '
 'siento vuestras dulces caricias en este costado!\n'
 '\n'
 'A lo cual la dama, sin perder el gracejo, y con una sonrisa pícara, le '
 'replicó al instante:\n'
 '\n'
 '—Pues, señor mío, si por un palo no sentís, ruego a Dios que el otro no os '
 'sea de la misma madera, que entonces sí que sería desdicha mayor.\n'
 '\n'
 '¡Y con esto, señores, os dejo 

Existen múltiples esquemas de cómo podemos articular estas instrucciones pero debemos ante todo contemplar que el modelo no tiene memoria y su conocimiento está acotado, de forma que deberemos informar en cada interacción de:

* **Rol** del modelo. Idioma y consideraciones que debe tener a la hora de resolver la tarea.
* **Contexto** a modo de información complementaria, ejemplo o forma en la que proceder para resolver la petición.
* **Instrucción** o tarea que vaya a realizar (resume este texto, genera la documentación de este código, etc.)
* **Formato** o manera en que la respuesta será provista (texto plano, formato concreto - JSON, HTML, ...)